disability fitness training exercises

In [1]:
import cv2
import mediapipe as mp

# =========================
#  Initialize MediaPipe Pose
# =========================
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# ---------------------------------------------------
# Helper function to detect pose on a given video frame
# ---------------------------------------------------
def detect_pose(frame):
    # Convert frame to RGB for MediaPipe processing
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run pose detection
    with mp_pose.Pose(static_image_mode=False,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:
        results = pose.process(frame_rgb)

        # Draw landmarks if detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    return frame


# ---------------------------------------------------
# Function to run pose detection + reference video side by side
# ---------------------------------------------------
def run_exercise(reference_path, exercise_name):
    # Open webcam and reference video
    webcam = cv2.VideoCapture(0)
    reference = cv2.VideoCapture(reference_path)

    if not reference.isOpened():
        print(f"Error: Could not open reference video for {exercise_name}.")
        return

    print(f"\nStarting {exercise_name}... (Press 'q' to quit)")

    while True:
        ret1, live_frame = webcam.read()
        ret2, ref_frame = reference.read()

        # Restart the reference video if it ends
        if not ret2:
            reference.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        # If webcam fails, stop
        if not ret1:
            print("Error: Webcam not detected.")
            break

        # Run pose detection on the live webcam feed
        live_frame = detect_pose(live_frame)

        # Resize both frames for display
        live_frame = cv2.resize(live_frame, (640, 480))
        ref_frame = cv2.resize(ref_frame, (640, 480))

        # Combine frames horizontally
        combined = cv2.hconcat([live_frame, ref_frame])

        # Display the combined view
        cv2.imshow(f"{exercise_name} - Live Pose vs Reference Video", combined)

        # Press 'q' to quit the current exercise
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    webcam.release()
    reference.release()
    cv2.destroyAllWindows()


# ---------------------------------------------------
# Exercise selection menu
# ---------------------------------------------------
def main():
    while True:
        print("\n========= Fitoproto Exercise Menu =========")
        print("1. Bicep Curl")
        print("2. Squat")
        print("3. Push-up")
        print("4. Exit")
        print("===========================================")

        choice = input("Select an exercise (1-4): ")

        # Change these paths to your own exercise videos
        if choice == '1':
            run_exercise("videos/bicep_curl.mp4", "Bicep Curl")
        elif choice == '2':
            run_exercise("videos/squat.mp4", "Squat")
        elif choice == '3':
            run_exercise("videos/pushup.mp4", "Push-up")
        elif choice == '4':
            print("Exiting Fitoproto. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1–4.")


# ---------------------------------------------------
# Entry point
# ---------------------------------------------------
if __name__ == "__main__":
    main()



========= Fitoproto Exercise Menu =========
1. Bicep Curl
2. Squat
3. Push-up
4. Exit
Error: Could not open reference video for Bicep Curl.

========= Fitoproto Exercise Menu =========
1. Bicep Curl
2. Squat
3. Push-up
4. Exit
Error: Could not open reference video for Squat.

========= Fitoproto Exercise Menu =========
1. Bicep Curl
2. Squat
3. Push-up
4. Exit
Exiting Fitoproto. Goodbye!


In [ ]:
!pip install ipykernel

: 

In [3]:
"""
pose_detection.py

Live pose detection using MediaPipe and OpenCV.

Features:
- Runs live webcam or processes a video file
- Draws pose landmarks and connections
- Computes joint angles
- Simple rep counting for bicep curls and squats (configurable)
- Saves annotated output video if --output is provided

Requirements:
- Python 3.8+
- pip install opencv-python mediapipe numpy

Usage examples:
python pose_detection.py                # open webcam
python pose_detection.py --source video.mp4 --output out.mp4
python pose_detection.py --source 0 --exercise bicep --display-angle

"""

import cv2
import mediapipe as mp
import numpy as np
import argparse
import time
from collections import deque

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def calculate_angle(a, b, c):
    """Calculate angle (in degrees) formed at point b by points a-b-c.
    a, b, c: (x, y) tuples or numpy arrays in the same coordinate system.
    Returns angle in range [0, 180].
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    # Normalize
    # Protect against zero-length vectors
    def safe_norm(v):
        norm = np.linalg.norm(v)
        return norm if norm != 0 else 1e-6

    cos_angle = np.dot(ba, bc) / (safe_norm(ba) * safe_norm(bc))
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle = np.degrees(np.arccos(cos_angle))
    return angle


class RepCounter:
    """Simple finite-state rep counter for one-dimensional exercises using a single joint angle.

    state machine approach:
      - detects transitions from 'down'->'up' or vice-versa
      - counts a rep when defined threshold is crossed twice (full cycle)

    Use for exercises like bicep curls (elbow angle) or squats (knee angle).
    """

    def __init__(self, up_threshold=160, down_threshold=40, smoothing=5):
        self.up_threshold = up_threshold
        self.down_threshold = down_threshold
        self.smoothing = smoothing
        self.angle_history = deque(maxlen=smoothing)
        self.count = 0
        self.stage = None  # 'up' or 'down' or None

    def update(self, angle):
        self.angle_history.append(angle)
        avg_angle = np.mean(self.angle_history)

        # determine stage
        if avg_angle > self.up_threshold:
            # arm fully extended / standing
            if self.stage == 'down' or self.stage is None:
                # moved to up
                self.stage = 'up'
                # don't count immediately; wait for full cycle
        elif avg_angle < self.down_threshold:
            # arm fully contracted / squatted
            if self.stage == 'up' or self.stage is None:
                # moved to down
                self.stage = 'down'
                # increment count when we reach 'down' from 'up'
                # For a more conservative count, count when returning to 'up' instead.
                self.count += 1
        return avg_angle


def extract_landmark_coords(landmarks, image_shape, landmark_index):
    """Return (x, y) pixel coordinates of a landmark index or None if not present."""
    if landmarks is None:
        return None
    l = landmarks.landmark[landmark_index]
    h, w = image_shape[:2]
    return (int(l.x * w), int(l.y * h))


def draw_angle_text(image, point, angle, offset=(0, -20)):
    x, y = point
    cv2.putText(
        image,
        f"{int(angle)}\u00b0",
        (x + offset[0], y + offset[1]),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (255, 255, 255),
        2,
        cv2.LINE_AA,
    )


def main(args):
    source = args.source
    exercise = args.exercise
    show_angle = args.display_angle

    # Input source handling: allow integer webcam index
    try:
        source_id = int(source)
    except Exception:
        source_id = source

    cap = cv2.VideoCapture(source_id)
    if not cap.isOpened():
        print(f"Error: cannot open source {source}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

    out_writer = None
    if args.output:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_writer = cv2.VideoWriter(args.output, fourcc, fps, (width, height))

    # Choose thresholds depending on exercise
    if exercise == 'bicep':
        # elbow angle: shoulder - elbow - wrist
        rep_counter = RepCounter(up_threshold=150, down_threshold=45, smoothing=7)
        monitor = 'elbow'
    elif exercise == 'squat':
        # knee angle: hip - knee - ankle
        rep_counter = RepCounter(up_threshold=165, down_threshold=100, smoothing=7)
        monitor = 'knee'
    else:
        rep_counter = None
        monitor = None

    # MediaPipe Pose
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    )

    prev_time = 0

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            image.flags.writeable = True
            annotated_image = frame.copy()

            # Draw pose landmarks
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    annotated_image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                )

            # If exercise monitoring enabled, compute angle and update counter
            display_info = {}
            if rep_counter is not None and results.pose_landmarks:
                lm = results.pose_landmarks
                h, w = annotated_image.shape[:2]

                if monitor == 'elbow':
                    # left elbow angle
                    left_shoulder = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.LEFT_SHOULDER.value)
                    left_elbow = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.LEFT_ELBOW.value)
                    left_wrist = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.LEFT_WRIST.value)

                    # prefer right side if left is missing
                    right_shoulder = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.RIGHT_SHOULDER.value)
                    right_elbow = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.RIGHT_ELBOW.value)
                    right_wrist = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.RIGHT_WRIST.value)

                    if left_elbow and left_shoulder and left_wrist:
                        angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                        avg_angle = rep_counter.update(angle)
                        display_info['angle'] = avg_angle
                        draw_angle_text(annotated_image, left_elbow, avg_angle)
                    elif right_elbow and right_shoulder and right_wrist:
                        angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                        avg_angle = rep_counter.update(angle)
                        display_info['angle'] = avg_angle
                        draw_angle_text(annotated_image, right_elbow, avg_angle)

                elif monitor == 'knee':
                    left_hip = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.LEFT_HIP.value)
                    left_knee = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.LEFT_KNEE.value)
                    left_ankle = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.LEFT_ANKLE.value)

                    right_hip = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.RIGHT_HIP.value)
                    right_knee = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.RIGHT_KNEE.value)
                    right_ankle = extract_landmark_coords(lm, annotated_image.shape, mp_pose.PoseLandmark.RIGHT_ANKLE.value)

                    if left_knee and left_hip and left_ankle:
                        angle = calculate_angle(left_hip, left_knee, left_ankle)
                        avg_angle = rep_counter.update(angle)
                        display_info['angle'] = avg_angle
                        draw_angle_text(annotated_image, left_knee, avg_angle)
                    elif right_knee and right_hip and right_ankle:
                        angle = calculate_angle(right_hip, right_knee, right_ankle)
                        avg_angle = rep_counter.update(angle)
                        display_info['angle'] = avg_angle
                        draw_angle_text(annotated_image, right_knee, avg_angle)

            # Overlay rep count and FPS
            if rep_counter is not None:
                cv2.rectangle(annotated_image, (0, 0), (220, 80), (0, 0, 0), -1)
                cv2.putText(annotated_image, f"Exercise: {exercise}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                cv2.putText(annotated_image, f"Reps: {rep_counter.count}", (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                if 'angle' in display_info and show_angle:
                    cv2.putText(annotated_image, f"Angle: {int(display_info['angle'])}\u00b0", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            # FPS
            curr_time = time.time()
            fps_text = 1 / (curr_time - prev_time) if prev_time else 0
            prev_time = curr_time
            cv2.putText(annotated_image, f"FPS: {int(fps_text)}", (width - 100, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

            cv2.imshow('Pose Detection', annotated_image)

            if out_writer is not None:
                out_writer.write(annotated_image)

            key = cv2.waitKey(1) & 0xFF
            if key == 27 or key == ord('q'):
                break

    finally:
        cap.release()
        if out_writer is not None:
            out_writer.release()
        cv2.destroyAllWindows()
        pose.close()


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Live pose detection with simple rep counting')
    parser.add_argument('--source', type=str, default='0', help='Video source. Use integer for webcam (default 0) or path to video file')
    parser.add_argument('--output', type=str, default=None, help='Path to save annotated video (mp4)')
    parser.add_argument('--exercise', type=str, default=None, choices=[None, 'bicep', 'squat'], help='Exercise to monitor for rep counting')
    parser.add_argument('--display-angle', dest='display_angle', action='store_true', help='Overlay angle value at monitored joint')
    args = parser.parse_args()

    main(args)


usage: ipykernel_launcher.py [-h] [--source SOURCE] [--output OUTPUT]
                             [--exercise {None,bicep,squat}] [--display-angle]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\dewan\AppData\Roaming\jupyter\runtime\kernel-v3deba9b1d44d05c427f4e6c6e8d911d847d8a0b2b.json


SystemExit: 2

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime
import time


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    
    # Check if the file exists, if not, add headers
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists, no need to write headers
    
    # Append new data
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])
    
    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")

# to close the program much better 
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()  # Immediate program termination


# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Get video frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = 20  # Adjust FPS as needed

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Use 'MP4V' for .mp4
out = cv2.VideoWriter("output.mp4", fourcc, fps, (frame_width, frame_height))
# open the webcam
cap = cv2.VideoCapture(0)

# def draw_concentration_bar(image, count):
#     bar_height = int(368 * (count % 12) / 10)  # Simple visualization based on reps
#     cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
#     cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

def select_exercise():
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl()
    elif choice == "2":
        squat()
    elif choice == "3":
        pushup()
    else:
        print("Invalid choice. Please restart and enter a valid number.")

# to fix the issue in to much quick response
#  Updated Code with Delayed Feedback Switch
# Initialize smoothing variables

smoothed_fill_ratio = 0  # Store the smoothed value
alpha = 0.1  # Smoothing factor (Higher = Faster updates, Lower = Smoother)

def draw_concentration_bar(image, angle):
    """
    Draws a smoothed concentration bar based on squat depth.
    Uses Exponential Moving Average (EMA) for smooth transitions.
    """

    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill ratio (160° → 0%, 80° → 100%)
    target_fill_ratio = max(0, min(1, (160 - angle) / 80))  

    # Apply smoothing (Exponential Moving Average)
    # Adjust smoothing factor to make bar update faster
    alpha = 0.25  # Increased from 0.1 to 0.25 for a quicker response
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)  

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

# Global variable for bar smoothing
smoothed_fill_ratio = 0  

def draw_concentration_bar_biceps(image, angle):
    """Draws a smoothed concentration bar based on curl depth using EMA."""
    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  
    bar_y = 100  
    bar_height = 200  
    bar_width = 30  

    # Normalize angle to fill ratio (150° → 0%, 30° → 100%)
    target_fill_ratio = max(0, min(1, (150 - angle) / 120))  

    # Apply smoothing (Exponential Moving Average)
    alpha = 0.25  # Smoothing factor
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not curled enough)
    elif 70 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent curl)
    else:
        color = (0, 255, 0)  # Green (Perfect curl)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Curl", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def bicep_curl():
    """Tracks bicep curls and counts reps."""
    count = 0
    position = "down"

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Get keypoints for right arm
                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    
                    # add key pint for left also 
                    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    #right angle 
                    r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                    #left angle
                    l_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)

                    # DEBUG: Print values
                    print(f"Angle: {r_angle:.2f}, Position: {position}")

                    # Check bicep curl position logic 
                    #for both left and right hand
                    if r_angle > 140 and l_angle > 140:
                        position = "down"

                    if r_angle < 50 and l_angle < 50 and position == "down":
                        position = "up"
                        count += 1
                        print(f"✅ Curl Counted! Total: {count}")  

                    # Provide feedback
                    if r_angle > 140 and l_angle > 140:
                        feedback = "Extend your arm!"
                    elif r_angle < 50 :
                        feedback = "Full curl!"
                    else:
                        feedback = "Good form!"

                    # Display feedback
                    cv2.putText(image, f"Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    # Draw the concentration bar
                    draw_concentration_bar_biceps(image,  (r_angle + l_angle) / 2)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Bicep Curl", image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Bicep curl",count)
                cleanup()
                break

    # cap.release()
    # cv2.destroyAllWindows()


def draw_concentration_bar_squat(image, angle):
    """
    Draws a concentration bar on the screen based on the squat depth (angle).
    - More filled when squatting lower.
    - Color changes based on depth.
    """

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position of the bar
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill the bar
    fill_ratio = max(0, min(1, (160 - angle) / 80))  # 160° (empty) → 80° (full)
    fill_height = int(fill_ratio * bar_height)  # Compute filled height

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw text labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def squat():
    count = 0
    position = "up"
    shoulder_initial_y = None
    last_feedback = "Start your exercise"
    last_feedback_time = time.time()
    feedback_delay = 1.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

                angle = calculate_angle(hip, knee, ankle)

                if position == "up" and shoulder_initial_y is None:
                    shoulder_initial_y = shoulder[1]

                print(f"Angle: {angle:.2f}, Position: {position}")

                if angle > 160:
                    position = "up"
                    shoulder_initial_y = shoulder[1]

                if angle < 90 and position == "up" and shoulder[1] > shoulder_initial_y + 0.02:
                    position = "down"
                    count += 1
                    print(f"✅ Squat Counted! Total: {count}")

                new_feedback = last_feedback
                if angle > 160:
                    new_feedback = "Stand tall!"
                elif angle < 90:
                    new_feedback = "Squat low!"
                else:
                    new_feedback = "Good depth!"

                if time.time() - last_feedback_time > feedback_delay:
                    last_feedback = new_feedback
                    last_feedback_time = time.time()

                # Draw smoothed concentration bar
                draw_concentration_bar_squat(image, angle)

                cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, last_feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Squat", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Squats",count)
            cleanup()
            break

    # cap.release()
    # cv2.destroyAllWindows()


## solved by adding the shouder points 
## wrong counting


def pushup():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                angle = calculate_angle(shoulder, elbow, wrist)

                if angle > 160:
                    position = "up"
                if angle < 90 and position == "up":
                    position = "down"
                    count += 1
                
                feedback = "Good form!" if 80 < angle < 160 else "Keep your core tight!"
                
                cv2.putText(image, f"Pushup Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        draw_concentration_bar(image, count)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Pushup", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Pushup", count)
            cleanup()


select_exercise()


## Imports


In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime
import time


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    
    # Check if the file exists, if not, add headers
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists, no need to write headers
    
    # Append new data
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])
    
    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")

# to close the program much better 
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()  # Immediate program termination


# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Get video frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = 20  # Adjust FPS as needed

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Use 'MP4V' for .mp4
out = cv2.VideoWriter("output.mp4", fourcc, fps, (frame_width, frame_height))
# open the webcam
cap = cv2.VideoCapture(0)

# def draw_concentration_bar(image, count):
#     bar_height = int(368 * (count % 12) / 10)  # Simple visualization based on reps
#     cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
#     cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

def select_exercise():
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl()
    elif choice == "2":
        squat()
    elif choice == "3":
        pushup()
    else:
        print("Invalid choice. Please restart and enter a valid number.")


## Trial 1

#### Pushups


In [ ]:
def pushup():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                angle = calculate_angle(shoulder, elbow, wrist)

                if angle > 160:
                    position = "up"
                if angle < 90 and position == "up":
                    position = "down"
                    count += 1
                
                feedback = "Good form!" if 80 < angle < 160 else "Keep your core tight!"
                
                cv2.putText(image, f"Pushup Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        draw_concentration_bar(image, angle)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Pushup", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Pushup", count)
            cleanup()


smoothed_fill_ratio = 0  # Store the smoothed value
alpha = 0.1  # Smoothing factor (Higher = Faster updates, Lower = Smoother)

def draw_concentration_bar(image, angle):
    """
    Draws a smoothed concentration bar based on squat depth.
    Uses Exponential Moving Average (EMA) for smooth transitions.
    """

    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill ratio (160° → 0%, 80° → 100%)
    target_fill_ratio = max(0, min(1, (160 - angle) / 80))  

    # Apply smoothing (Exponential Moving Average)
    # Adjust smoothing factor to make bar update faster
    alpha = 0.25  # Increased from 0.1 to 0.25 for a quicker response
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)  

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

# Global variable for bar smoothing
smoothed_fill_ratio = 0  

pushup()




✅ Saved 1 reps of Pushup to exercise_log.csv


: 

### Full Hand Rotation

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime
import time


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    
    # Check if the file exists, if not, add headers
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists, no need to write headers
    
    # Append new data
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])
    
    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")

# to close the program much better 
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()  # Immediate program termination


# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Get video frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = 20  # Adjust FPS as needed

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Use 'MP4V' for .mp4
out = cv2.VideoWriter("output.mp4", fourcc, fps, (frame_width, frame_height))
# open the webcam
cap = cv2.VideoCapture(0)

# def draw_concentration_bar(image, count):
#     bar_height = int(368 * (count % 12) / 10)  # Simple visualization based on reps
#     cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
#     cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

def select_exercise():
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl()
    elif choice == "2":
        squat()
    elif choice == "3":
        pushup()
    else:
        print("Invalid choice. Please restart and enter a valid number.")

def draw_concentration_bar_rotation(image, angle_progress):
    """
    Draws a circular progress bar showing arm rotation completion (0–360°).
    angle_progress is the current rotation angle in degrees.
    """
    center = (100, 200)   # position of circle
    radius = 60
    thickness = 10

    # Map angle to progress 0–360
    progress_angle = int(angle_progress % 360)

    # Draw outer circle (gray background)
    cv2.circle(image, center, radius, (200, 200, 200), thickness)

    # Draw progress arc (green)
    cv2.ellipse(image, center, (radius, radius),
                0, -90, -90 + progress_angle, (0, 255, 0), thickness)

    # Display numeric progress inside the circle
    cv2.putText(image, f"{int(progress_angle)}°",
                (center[0] - 25, center[1] + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Add label
    cv2.putText(image, "Rotation Progress",
                (center[0] - 80, center[1] + 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)



def full_hand_rotation():
    count = 0
    position = None
    rotation_started = False

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Get keypoints for right arm
                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                               landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

                    # Calculate angles
                    # shoulder_elbow_wrist = calculate_angle(r_shoulder, r_elbow, r_wrist)
                    torso_shoulder_wrist = calculate_angle(
                        hip,
                        r_shoulder,
                        r_wrist
                    )

                    # DEBUG: show angles
                    print(f"Shoulder Rotation: {torso_shoulder_wrist:.2f}")

                    # Detect rotation based on shoulder-wrist angle changes
                    rotation_angle = calculate_rotation_angle(r_shoulder, r_wrist)

                    # Track rotations
                    if position is None:
                        position = rotation_angle
                        start_angle = rotation_angle
                        completed_rotation = False

                    # Detect full 360° rotation
                    angle_diff = rotation_angle - position
                    if angle_diff < -180:  # handle wrap-around
                        angle_diff += 360
                    elif angle_diff > 180:
                        angle_diff -= 360

                    position = rotation_angle

                     # Visual marker or progress bar (optional)
                    draw_concentration_bar_rotation(image, torso_shoulder_wrist)

                    # Accumulate total rotation
                    if abs(angle_diff) > 0:
                        total_angle = (total_angle + abs(angle_diff)) % 360
                        draw_concentration_bar_rotation(image, total_angle)

                        if not completed_rotation and total_angle > 350:
                            count += 1
                            completed_rotation = True
                            print(f"✅ Full Rotation Counted! Total: {count}")
                        elif total_angle < 20:
                            completed_rotation = False

                        # Feedback
                        if rotation_started:
                            feedback = "Keep rotating!"
                        else:
                            feedback = "Start arm circles!"
                        # Display feedback and count
                        cv2.putText(image, f"Rotations: {count}", (30, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        cv2.putText(image, feedback, (30, 80),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                       

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Full Hand Rotation", image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Full hand rotation", count)
                cleanup()
                break

full_hand_rotation()

























Shoulder Rotation: 11.09
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 12.01
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 13.75
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 14.04
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 14.91
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 14.87
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 13.44
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 13.80
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 15.26
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 14.29
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 14.52
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 14.70
Error: name 'calculate_rotation_angle' is not defined
Shoulder Rotation: 14.52
Error: name 'calculate_rota

KeyboardInterrupt: 

## Full Hand Rotations


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime
import time


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"

    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File exists, skip header writing

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


# Function to clean up camera and window
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle


# Function to calculate wrist rotation angle relative to shoulder
def calculate_rotation_angle(shoulder, wrist):
    """
    Calculates angle of wrist around the shoulder (in degrees 0–360).
    Used for circular motion tracking.
    """
    dx = wrist[0] - shoulder[0]
    dy = wrist[1] - shoulder[1]
    angle = np.degrees(np.arctan2(dy, dx))
    if angle < 0:
        angle += 360
    return angle


# Draw circular progress bar
def draw_concentration_bar_rotation(image, angle_progress):
    """
    Draws a circular progress bar showing arm rotation completion (0–360°).
    """
    center = (100, 200)
    radius = 60
    thickness = 10

    progress_angle = int(angle_progress % 360)

    # Background circle
    cv2.circle(image, center, radius, (80, 80, 80), thickness)

    # Progress arc
    cv2.ellipse(image, center, (radius, radius),
                0, -90, -90 + progress_angle, (0, 255, 0), thickness)

    # Text inside the circle
    cv2.putText(image, f"{int(progress_angle)}°",
                (center[0] - 25, center[1] + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Label below
    cv2.putText(image, "Rotation Progress",
                (center[0] - 80, center[1] + 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Open camera
cap = cv2.VideoCapture(0)


def full_hand_rotation():
    count = 0
    position = None
    total_angle = 0
    completed_rotation = False

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    # Calculate wrist rotation angle
                    rotation_angle = calculate_rotation_angle(r_shoulder, r_wrist)

                    if position is None:
                        position = rotation_angle
                        total_angle = 0

                    # Compute angle difference
                    angle_diff = rotation_angle - position
                    if angle_diff < -180:
                        angle_diff += 360
                    elif angle_diff > 180:
                        angle_diff -= 360

                    position = rotation_angle
                    total_angle = (total_angle + abs(angle_diff)) % 360

                    # Draw progress bar
                    draw_concentration_bar_rotation(image, total_angle)

                    # Detect full rotation
                    if not completed_rotation and total_angle > 350:
                        count += 1
                        completed_rotation = True
                        print(f"✅ Full Rotation Counted! Total: {count}")
                    elif total_angle < 20:
                        completed_rotation = False

                    # Display info
                    cv2.putText(image, f"Rotations: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            except Exception as e:
                print(f"Error: {e}")

            # Draw skeleton
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Full Hand Rotation", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Full hand rotation", count)
                cleanup()
                break


# Run
full_hand_rotation()


✅ Full Rotation Counted! Total: 1
✅ Full Rotation Counted! Total: 2
✅ Saved 2 reps of Full hand rotation to exercise_log.csv


: 

## Full Hand Rotation

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"

    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle


# Draw circular progress bar
def draw_concentration_bar_rotation(image, angle_value, center, label):
    """
    Draws a circular progress bar showing the current joint angle (0–180°).
    """
    radius = 60
    thickness = 10

    # Map the 0–180 angle to a 0–360 arc for smoother visualization
    progress_angle = int((angle_value / 180) * 360)

    # Background circle
    cv2.circle(image, center, radius, (80, 80, 80), thickness)

    # Progress arc (dynamic color)
    color = (0, int((angle_value / 180) * 255), 255 - int((angle_value / 180) * 255))
    cv2.ellipse(image, center, (radius, radius),
                0, -90, -90 + progress_angle, color, thickness)

    # Text inside circle
    cv2.putText(image, f"{int(angle_value)}°",
                (center[0] - 25, center[1] + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Label below
    cv2.putText(image, label,
                (center[0] - 60, center[1] + 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Open camera
cap = cv2.VideoCapture(0)


def full_hand_rotation():
    count = 0
    direction = 0  # 0 = going down, 1 = going up

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    # Get key points
                    r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                    # Calculate shoulder rotation (Hip–Shoulder–Wrist)
                    r_angle = calculate_angle(r_hip, r_shoulder, r_wrist)
                    l_angle = calculate_angle(l_hip, l_shoulder, l_wrist)

                    # Draw progress bars for each arm
                    draw_concentration_bar_rotation(image, r_angle, (100, 200), "Right Shoulder")
                    draw_concentration_bar_rotation(image, l_angle, (300, 200), "Left Shoulder")

                    # Rep counting logic (based on up and down arm motion)
                    if r_angle > 150 or l_angle > 150:
                        direction = 1  # Arm fully up
                    if (r_angle < 50 or l_angle < 50) and direction == 1:
                        count += 1
                        direction = 0
                        print(f"✅ Full Rotation Counted! Total: {count}")

                    # Display feedback
                    cv2.putText(image, f"Rotations: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # Draw landmarks
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("Full Hand Rotation (Hip–Shoulder–Wrist Angle)", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Full hand rotation", count)
                cleanup()
                break


# Run the function
full_hand_rotation()


✅ Full Rotation Counted! Total: 1
✅ Full Rotation Counted! Total: 2
✅ Full Rotation Counted! Total: 3
✅ Full Rotation Counted! Total: 4
✅ Full Rotation Counted! Total: 5
✅ Full Rotation Counted! Total: 6
✅ Full Rotation Counted! Total: 7
✅ Full Rotation Counted! Total: 8
✅ Full Rotation Counted! Total: 9
✅ Full Rotation Counted! Total: 10
✅ Full Rotation Counted! Total: 11
✅ Full Rotation Counted! Total: 12
✅ Full Rotation Counted! Total: 13
✅ Full Rotation Counted! Total: 14
✅ Full Rotation Counted! Total: 15
✅ Full Rotation Counted! Total: 16
✅ Full Rotation Counted! Total: 17
✅ Full Rotation Counted! Total: 18
✅ Full Rotation Counted! Total: 19
✅ Full Rotation Counted! Total: 20
✅ Full Rotation Counted! Total: 21
✅ Full Rotation Counted! Total: 22
✅ Full Rotation Counted! Total: 23
✅ Full Rotation Counted! Total: 24
✅ Full Rotation Counted! Total: 25
✅ Full Rotation Counted! Total: 26
✅ Full Rotation Counted! Total: 27
✅ Full Rotation Counted! Total: 28
✅ Full Rotation Counted! Tota

: 

## External Rotation


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime


# Log reps to CSV
import csv
import datetime

def log_reps(exercise_name, count):
    filename = "exercise_log.csv"

    # Create the CSV file with headers if it doesn't exist
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass

    # Append new data to the CSV file
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        now = datetime.datetime.now()
        date = now.strftime("%Y-%m-%d")
        time = now.strftime("%H:%M:%S")
        writer.writerow([date, time, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to {filename}")



def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# Draw rectangular bar
def draw_progress_bar(image, progress, position, label):
    """
    Draws a rectangular progress bar showing progress (0–100%).
    """
    x, y = position
    bar_width = 30
    bar_height = 200
    filled_height = int((progress / 100) * bar_height)

    # Background bar
    cv2.rectangle(image, (x, y - bar_height), (x + bar_width, y), (50, 50, 50), 2)
    # Filled section
    color = (0, int(progress * 2.55), 255 - int(progress * 2.55))
    cv2.rectangle(image, (x, y - filled_height), (x + bar_width, y), color, -1)

    # Label and percentage
    cv2.putText(image, label, (x - 20, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, f"{int(progress)}%", (x - 10, y - bar_height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Open camera
cap = cv2.VideoCapture(0)


def shoulder_external_rotation_distance():
    count = 0
    direction = 0  # 0 = inward, 1 = outward

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)
            image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    # Key points
                    r_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
                    r_hip_opposite = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    l_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
                    l_hip_opposite = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

                    # Calculate distance between wrist and opposite hip
                    r_dist = np.sqrt((r_wrist.x - r_hip_opposite.x) ** 2 + (r_wrist.y - r_hip_opposite.y) ** 2)
                    l_dist = np.sqrt((l_wrist.x - l_hip_opposite.x) ** 2 + (l_wrist.y - l_hip_opposite.y) ** 2)

                    # Normalize distance to percentage scale
                    r_progress = np.interp(r_dist, [0.15, 0.35], [0, 100])  # Adjust range as needed
                    l_progress = np.interp(l_dist, [0.15, 0.35], [0, 100])
                    r_progress = np.clip(r_progress, 0, 100)
                    l_progress = np.clip(l_progress, 0, 100)

                    # Draw progress bars
                    draw_progress_bar(image, r_progress, (80, 400), "Right Arm")
                    draw_progress_bar(image, l_progress, (140, 400), "Left Arm")

                    # Rep counting logic
                    if r_progress > 80 or l_progress > 80:
                        direction = 1  # outward
                    if (r_progress < 30 or l_progress < 30) and direction == 1:
                        count += 1
                        direction = 0
                        print(f"✅ External Rotation Counted! Total: {count}")

                    # Display feedback
                    cv2.putText(image, f"Rotations: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # Draw landmarks
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("Shoulder External Rotation (Distance-Based)", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Shoulder External Rotation (Distance)", count)
                cleanup()
                break


# Run function
shoulder_external_rotation_distance()


✅ External Rotation Counted! Total: 1
✅ External Rotation Counted! Total: 2
✅ External Rotation Counted! Total: 3
✅ External Rotation Counted! Total: 4
✅ External Rotation Counted! Total: 5
✅ External Rotation Counted! Total: 6
✅ External Rotation Counted! Total: 7
✅ External Rotation Counted! Total: 8
✅ External Rotation Counted! Total: 9
✅ External Rotation Counted! Total: 10
✅ External Rotation Counted! Total: 11
✅ External Rotation Counted! Total: 12
✅ External Rotation Counted! Total: 13
✅ External Rotation Counted! Total: 14
✅ External Rotation Counted! Total: 15
✅ External Rotation Counted! Total: 16
✅ External Rotation Counted! Total: 17
✅ External Rotation Counted! Total: 18
✅ External Rotation Counted! Total: 19
✅ External Rotation Counted! Total: 20
✅ External Rotation Counted! Total: 21
✅ External Rotation Counted! Total: 22
✅ External Rotation Counted! Total: 23
✅ External Rotation Counted! Total: 24
✅ External Rotation Counted! Total: 25
✅ External Rotation Counted! Total

: 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"

    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle


# Draw circular progress bar
def draw_concentration_bar_rotation(image, angle_value, center, label):
    """
    Draws a circular progress bar showing the current rotation angle (0–180°).
    """
    radius = 60
    thickness = 10
    progress_angle = int((angle_value / 180) * 360)

    # Dynamic color: red -> green as angle increases
    color = (0, int((angle_value / 180) * 255), 255 - int((angle_value / 180) * 255))

    # Background
    cv2.circle(image, center, radius, (80, 80, 80), thickness)
    # Progress arc
    cv2.ellipse(image, center, (radius, radius),
                0, -90, -90 + progress_angle, color, thickness)

    # Degree text
    cv2.putText(image, f"{int(angle_value)}°",
                (center[0] - 25, center[1] + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Label
    cv2.putText(image, label,
                (center[0] - 70, center[1] + 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Open camera
cap = cv2.VideoCapture(0)


def shoulder_external_rotation():
    count = 0
    direction = 0  # 0 = inward rotation, 1 = outward rotation

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    # Get key points
                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                    # Calculate external rotation (shoulder–elbow–wrist)
                    r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                    l_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)

                    # Draw circular progress bars
                    draw_concentration_bar_rotation(image, r_angle, (100, 200), "Right Rotation")
                    draw_concentration_bar_rotation(image, l_angle, (300, 200), "Left Rotation")

                    # Rep logic (outward + inward cycle)
                    if r_angle > 100 or l_angle > 100:
                        direction = 1  # outward rotation
                    if (r_angle < 40 or l_angle < 40) and direction == 1:
                        count += 1
                        direction = 0
                        print(f"✅ External Rotation Counted! Total: {count}")

                    # Display count
                    cv2.putText(image, f"Rotations: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # Draw skeleton
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("Shoulder External Rotation Tracker", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Shoulder External Rotation", count)
                cleanup()
                break


# Run it
shoulder_external_rotation()


✅ External Rotation Counted! Total: 1
✅ External Rotation Counted! Total: 2
✅ External Rotation Counted! Total: 3
✅ External Rotation Counted! Total: 4
✅ External Rotation Counted! Total: 5
✅ External Rotation Counted! Total: 6
✅ External Rotation Counted! Total: 7
✅ External Rotation Counted! Total: 8
✅ External Rotation Counted! Total: 9
✅ External Rotation Counted! Total: 10
✅ External Rotation Counted! Total: 11
✅ External Rotation Counted! Total: 12
✅ External Rotation Counted! Total: 13
✅ External Rotation Counted! Total: 14
✅ External Rotation Counted! Total: 15
✅ External Rotation Counted! Total: 16
✅ External Rotation Counted! Total: 17
✅ External Rotation Counted! Total: 18
✅ External Rotation Counted! Total: 19
✅ External Rotation Counted! Total: 20
✅ External Rotation Counted! Total: 21
✅ External Rotation Counted! Total: 22
✅ External Rotation Counted! Total: 23
✅ External Rotation Counted! Total: 24
✅ External Rotation Counted! Total: 25
✅ External Rotation Counted! Total

: 

## Adapted Burpees


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime


# ---------- Logging ----------
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


# ---------- Cleanup ----------
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# ---------- Draw Rectangular Progress Bar ----------
def draw_vertical_bar(image, progress, x, y, width, height, label):
    """
    Draws a rectangular bar (vertical fill).
    progress: 0.0–1.0
    """
    progress = np.clip(progress, 0, 1)

    # Background
    cv2.rectangle(image, (x, y), (x + width, y + height), (50, 50, 50), 2)

    # Progress height (fill from bottom to top)
    filled_height = int(height * progress)
    cv2.rectangle(image, (x, y + height - filled_height),
                  (x + width, y + height), (0, int(255 * progress), 255 - int(255 * progress)), -1)

    # Label
    cv2.putText(image, label, (x - 20, y + height + 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# ---------- MediaPipe Setup ----------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)


# ---------- Adapted Burpees Tracker ----------
def adapted_burpees():
    count = 0
    direction = 0  # 0 = going down, 1 = going up

    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    # Get keypoints for wrists and hips
                    r_wrist_y = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
                    l_wrist_y = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
                    r_hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
                    l_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
                    nose_y = landmarks[mp_pose.PoseLandmark.NOSE.value].y  # Head reference

                    # Average wrist position
                    wrist_y = (r_wrist_y + l_wrist_y) / 2
                    hip_y = (r_hip_y + l_hip_y) / 2

                    # Normalize progress: wrist near hip = 0, wrist near head = 1
                    total_range = hip_y - nose_y
                    progress = (hip_y - wrist_y) / total_range
                    progress = np.clip(progress, 0, 1)

                    # Draw progress bar
                    draw_vertical_bar(image, progress, 50, 100, 30, 300, "Arm Raise")

                    # Rep counting logic
                    if progress > 0.8:
                        direction = 1  # arms up
                    if progress < 0.3 and direction == 1:
                        count += 1
                        direction = 0
                        print(f"✅ Burpee Counted! Total: {count}")

                    # Display count
                    cv2.putText(image, f"Burpees: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # Draw skeleton
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("Adapted Burpees Tracker", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Adapted Burpees", count)
                cleanup()
                break


# ---------- Run ----------
adapted_burpees()


✅ Burpee Counted! Total: 1
✅ Burpee Counted! Total: 2
✅ Burpee Counted! Total: 3
✅ Burpee Counted! Total: 4
✅ Burpee Counted! Total: 5
✅ Burpee Counted! Total: 6
✅ Burpee Counted! Total: 7
✅ Burpee Counted! Total: 8
✅ Burpee Counted! Total: 9
✅ Burpee Counted! Total: 10
✅ Burpee Counted! Total: 11
✅ Burpee Counted! Total: 12
✅ Burpee Counted! Total: 13
✅ Burpee Counted! Total: 14
✅ Burpee Counted! Total: 15
✅ Burpee Counted! Total: 16
✅ Saved 16 reps of Adapted Burpees to exercise_log.csv


: 

## Seated wood chop

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime


# ---------- Logging ----------
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


# ---------- Cleanup ----------
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# ---------- Draw Horizontal Progress Bar ----------
def draw_horizontal_bar(image, progress, x, y, width, height, label):
    """
    Draws a horizontal rectangular bar.
    progress: 0.0–1.0
    """
    progress = np.clip(progress, 0, 1)

    # Outline
    cv2.rectangle(image, (x, y), (x + width, y + height), (50, 50, 50), 2)

    # Fill
    filled_width = int(width * progress)
    cv2.rectangle(image, (x, y),
                  (x + filled_width, y + height),
                  (0, int(255 * progress), 255 - int(255 * progress)), -1)

    # Label
    cv2.putText(image, label, (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# ---------- MediaPipe Setup ----------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)


# ---------- Seated Wood Chop Tracker ----------
def seated_wood_chop():
    count = 0
    direction = 0  # 0 = down-right, 1 = up-left

    with mp_pose.Pose(min_detection_confidence=0.6,
                      min_tracking_confidence=0.6) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    # Use wrists and hips to detect diagonal swing
                    r_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
                    l_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
                    r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

                    # Average wrist position
                    wrist_x = (r_wrist.x + l_wrist.x) / 2
                    wrist_y = (r_wrist.y + l_wrist.y) / 2

                    # Define start (upper left near shoulder) and end (lower right near hip)
                    start_x, start_y = l_shoulder.x, l_shoulder.y
                    end_x, end_y = r_hip.x, r_hip.y

                    # Compute normalized progress along diagonal
                    dx = end_x - start_x
                    dy = end_y - start_y
                    vector_length = np.sqrt(dx**2 + dy**2)

                    # Projection of wrist position along the diagonal
                    proj = ((wrist_x - start_x) * dx + (wrist_y - start_y) * dy) / (vector_length**2)
                    progress = np.clip(proj, 0, 1)

                    # Draw progress bar
                    draw_horizontal_bar(image, progress, 50, 100, 300, 30, "Diagonal Swing")

                    # Rep counting
                    if progress > 0.85:
                        direction = 1
                    if progress < 0.15 and direction == 1:
                        count += 1
                        direction = 0
                        print(f"✅ Wood Chop Counted! Total: {count}")

                    # Display count
                    cv2.putText(image, f"Wood Chops: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # Draw skeleton
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("Seated Wood Chop Tracker", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Seated Wood Chop", count)
                cleanup()
                break


# ---------- Run ----------
seated_wood_chop()


✅ Wood Chop Counted! Total: 1
✅ Wood Chop Counted! Total: 2
✅ Wood Chop Counted! Total: 3
✅ Wood Chop Counted! Total: 4
✅ Saved 4 reps of Seated Wood Chop to exercise_log.csv


: 

## In and Out

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime

# ---------- Logging ----------
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


# ---------- Cleanup ----------
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# ---------- Draw Horizontal Progress Bar ----------
def draw_horizontal_bar(image, progress, x, y, width, height, label):
    """
    Draws a horizontal rectangular bar.
    progress: 0.0–1.0
    """
    progress = np.clip(progress, 0, 1)

    # Outline
    cv2.rectangle(image, (x, y), (x + width, y + height), (50, 50, 50), 2)

    # Fill
    filled_width = int(width * progress)
    cv2.rectangle(image, (x, y),
                  (x + filled_width, y + height),
                  (0, int(255 * progress), 255 - int(255 * progress)), -1)

    # Label
    cv2.putText(image, label, (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# ---------- MediaPipe Setup ----------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)


# ---------- In & Out Arm Curl Tracker ----------
def in_and_out_curl():
    count = 0
    direction = 0  # 0 = extending, 1 = curling in

    with mp_pose.Pose(min_detection_confidence=0.6,
                      min_tracking_confidence=0.6) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    # Use right arm for tracking
                    r_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    r_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                    r_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
                    r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

                    # Ensure arm is roughly 90° from torso (static posture check)
                    shoulder_to_hip = np.sqrt(
                        (r_hip.x - r_shoulder.x) ** 2 + (r_hip.y - r_shoulder.y) ** 2)
                    shoulder_to_elbow = np.sqrt(
                        (r_elbow.x - r_shoulder.x) ** 2 + (r_elbow.y - r_shoulder.y) ** 2)

                    ratio = shoulder_to_elbow / shoulder_to_hip
                    # You could add a visual warning if not in 90° position

                    # Measure distance between wrist and shoulder
                    wrist_to_shoulder = np.sqrt(
                        (r_wrist.x - r_shoulder.x) ** 2 + (r_wrist.y - r_shoulder.y) ** 2)

                    # Normalize the distance (assuming min at curl, max at extension)
                    min_dist, max_dist = 0.10, 0.40  # tune these thresholds for accuracy
                    progress = np.clip((max_dist - wrist_to_shoulder) / (max_dist - min_dist), 0, 1)

                    # Draw progress bar
                    draw_horizontal_bar(image, progress, 50, 100, 300, 30, "In & Out Curl")

                    # Rep counting logic
                    if progress > 0.85:
                        direction = 1
                    if progress < 0.15 and direction == 1:
                        count += 1
                        direction = 0
                        print(f"✅ In & Out Curl Counted! Total: {count}")

                    # Display count
                    cv2.putText(image, f"In & Out Curls: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # Draw skeleton
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("In & Out Curl Tracker", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("In & Out Curl", count)
                cleanup()
                break


# ---------- Run ----------
in_and_out_curl()


✅ In & Out Curl Counted! Total: 1
✅ In & Out Curl Counted! Total: 2
✅ In & Out Curl Counted! Total: 3
✅ Saved 3 reps of In & Out Curl to exercise_log.csv


: 

## Seated Working in and out

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime

# ---------- Logging ----------
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


# ---------- Cleanup ----------
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# ---------- Draw Horizontal Progress Bar ----------
def draw_horizontal_bar(image, progress, x, y, width, height, label):
    progress = np.clip(progress, 0, 1)

    # Outline
    cv2.rectangle(image, (x, y), (x + width, y + height), (50, 50, 50), 2)

    # Fill
    filled_width = int(width * progress)
    cv2.rectangle(image, (x, y),
                  (x + filled_width, y + height),
                  (0, int(255 * progress), 255 - int(255 * progress)), -1)

    # Label
    cv2.putText(image, label, (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# ---------- Calculate angle between 3 points ----------
def calculate_angle(a, b, c):
    """Angle at point 'b' formed by points a, b, c"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle


# ---------- MediaPipe Setup ----------
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)


# ---------- Both Arm Curl Tracker ----------
def in_and_out_both_arms():
    count = 0
    direction = 0  # 0 = extending, 1 = curling in

    with mp_pose.Pose(min_detection_confidence=0.6,
                      min_tracking_confidence=0.6) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    lm = results.pose_landmarks.landmark

                    # --- Right Arm ---
                    r_shoulder = [lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_elbow = [lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                               lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    r_wrist = [lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    # --- Left Arm ---
                    l_shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                               lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    l_wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                               lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                    # Compute angles for both arms
                    r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                    l_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
                    avg_angle = (r_angle + l_angle) / 2  # track both arms together

                    # Normalize angle to progress bar
                    progress = np.interp(avg_angle, [50, 170], [1, 0])  # 50° = curl, 170° = straight

                    # Draw progress bar
                    draw_horizontal_bar(image, progress, 50, 100, 300, 30, "In & Out Both Arms")

                    # Rep counting logic
                    if progress > 0.85:
                        direction = 1
                    if progress < 0.15 and direction == 1:
                        count += 1
                        direction = 0
                        print(f"✅ Rep Counted! Total: {count}")

                    # Display data
                    cv2.putText(image, f"In & Out Reps: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(image, f"Angle (Avg): {int(avg_angle)}°", (30, 150),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                    # Draw landmarks
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("In & Out (Both Arms) Tracker", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("In & Out Both Arms", count)
                cleanup()
                break


# ---------- Run ----------
in_and_out_both_arms()


✅ Rep Counted! Total: 1
✅ Rep Counted! Total: 2
✅ Rep Counted! Total: 3
✅ Rep Counted! Total: 4
✅ Rep Counted! Total: 5
✅ Rep Counted! Total: 6
✅ Rep Counted! Total: 7
✅ Rep Counted! Total: 8
✅ Rep Counted! Total: 9
✅ Saved 9 reps of In & Out Both Arms to exercise_log.csv


: 

## Side Reach

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime

# ---------- Logging ----------
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


# ---------- Cleanup ----------
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# ---------- Draw Horizontal Progress Bar ----------
def draw_horizontal_bar(image, progress, x, y, width, height, label):
    progress = np.clip(progress, 0, 1)
    cv2.rectangle(image, (x, y), (x + width, y + height), (50, 50, 50), 2)
    filled_width = int(width * progress)
    cv2.rectangle(image, (x, y),
                  (x + filled_width, y + height),
                  (0, int(255 * progress), 255 - int(255 * progress)), -1)
    cv2.putText(image, label, (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# ---------- MediaPipe Setup ----------
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)


# ---------- Side Reach Tracker ----------
def side_reach_both_arms():
    count = 0
    right_dir = 0
    left_dir = 0

    with mp_pose.Pose(min_detection_confidence=0.6,
                      min_tracking_confidence=0.6) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    lm = results.pose_landmarks.landmark

                    # --- Right Arm ---
                    r_hip = [lm[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                             lm[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    r_shoulder = [lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_wrist = [lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    # --- Left Arm ---
                    l_hip = [lm[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                             lm[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    l_shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                               lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                    # Calculate vertical distances (hip to wrist)
                    r_dist = r_hip[1] - r_wrist[1]
                    l_dist = l_hip[1] - l_wrist[1]

                    # Normalize progress for each arm (hand moving up)
                    r_progress = np.interp(r_dist, [0.05, 0.35], [0, 1])
                    l_progress = np.interp(l_dist, [0.05, 0.35], [0, 1])

                    # Draw progress bars
                    draw_horizontal_bar(image, r_progress, 50, 100, 300, 30, "Right Hand Reach")
                    draw_horizontal_bar(image, l_progress, 50, 150, 300, 30, "Left Hand Reach")

                    # Rep counting logic (both arms up → both down)
                    if r_progress > 0.85 and l_progress > 0.85:
                        if right_dir == 0 and left_dir == 0:
                            right_dir = 1
                            left_dir = 1
                    if r_progress < 0.15 and l_progress < 0.15 and right_dir == 1 and left_dir == 1:
                        count += 1
                        right_dir = 0
                        left_dir = 0
                        print(f"✅ Rep Counted! Total: {count}")

                    # Display Data
                    cv2.putText(image, f"Side Reach Reps: {count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(image, f"R Dist: {r_dist:.2f}", (400, 120),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    cv2.putText(image, f"L Dist: {l_dist:.2f}", (400, 170),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                    # Draw Pose Landmarks
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("Side Reach (Both Hands) Tracker", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Side Reach (Both Hands)", count)
                cleanup()
                break


# ---------- Run ----------
side_reach_both_arms()


✅ Rep Counted! Total: 1
✅ Rep Counted! Total: 2
✅ Rep Counted! Total: 3
✅ Rep Counted! Total: 4
✅ Saved 4 reps of Side Reach (Both Hands) to exercise_log.csv


: 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime

# ---------- Logging ----------
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])

    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")


# ---------- Cleanup ----------
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()


# ---------- Draw Horizontal Progress Bar ----------
def draw_horizontal_bar(image, progress, x, y, width, height, label):
    progress = np.clip(progress, 0, 1)
    cv2.rectangle(image, (x, y), (x + width, y + height), (50, 50, 50), 2)
    filled_width = int(width * progress)
    cv2.rectangle(image, (x, y),
                  (x + filled_width, y + height),
                  (0, int(255 * progress), 255 - int(255 * progress)), -1)
    cv2.putText(image, label, (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# ---------- MediaPipe Setup ----------
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)


# ---------- Side Reach Tracker ----------
def side_reach_both_arms():
    total_count = 0
    right_count, left_count = 0, 0
    right_dir, left_dir = 0, 0  # 0 = down, 1 = up

    with mp_pose.Pose(min_detection_confidence=0.6,
                      min_tracking_confidence=0.6) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    lm = results.pose_landmarks.landmark

                    # --- Right Arm ---
                    r_hip = [lm[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                             lm[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    r_wrist = [lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    # --- Left Arm ---
                    l_hip = [lm[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                             lm[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    l_wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                               lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                    # Calculate vertical distances (hip to wrist)
                    r_dist = r_hip[1] - r_wrist[1]
                    l_dist = l_hip[1] - l_wrist[1]

                    # Normalize progress for each arm (hip middle -> straight up)
                    # Adjusted for better sensitivity
                    r_progress = np.interp(r_dist, [0.05, 0.45], [0, 1])
                    l_progress = np.interp(l_dist, [0.05, 0.45], [0, 1])

                    # Draw progress bars
                    draw_horizontal_bar(image, r_progress, 50, 100, 300, 30, "Right Hand Reach")
                    draw_horizontal_bar(image, l_progress, 50, 150, 300, 30, "Left Hand Reach")

                    # ----- RIGHT ARM -----
                    if r_progress > 0.85:
                        right_dir = 1  # Hand up
                    if r_progress < 0.15 and right_dir == 1:
                        right_count += 1
                        total_count += 1
                        right_dir = 0
                        print(f"✅ Right Arm Rep Counted! Total: {right_count}")

                    # ----- LEFT ARM -----
                    if l_progress > 0.85:
                        left_dir = 1  # Hand up
                    if l_progress < 0.15 and left_dir == 1:
                        left_count += 1
                        total_count += 1
                        left_dir = 0
                        print(f"✅ Left Arm Rep Counted! Total: {left_count}")

                    # Display Data
                    cv2.putText(image, f"Total Reps: {total_count}", (30, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(image, f"Right Reps: {right_count}", (400, 120),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    cv2.putText(image, f"Left Reps: {left_count}", (400, 170),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                    # Draw Pose Landmarks
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            except Exception as e:
                print(f"Error: {e}")

            cv2.imshow("Side Reach (Single Arm Counting)", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Side Reach (Single Arm Counting)", total_count)
                cleanup()
                break


# ---------- Run ----------
side_reach_both_arms()


✅ Left Arm Rep Counted! Total: 1
✅ Right Arm Rep Counted! Total: 1
✅ Left Arm Rep Counted! Total: 2
✅ Left Arm Rep Counted! Total: 3
✅ Left Arm Rep Counted! Total: 4
✅ Right Arm Rep Counted! Total: 2
✅ Left Arm Rep Counted! Total: 5
✅ Left Arm Rep Counted! Total: 6
✅ Right Arm Rep Counted! Total: 3
✅ Left Arm Rep Counted! Total: 7
✅ Left Arm Rep Counted! Total: 8
✅ Right Arm Rep Counted! Total: 4
✅ Left Arm Rep Counted! Total: 9
✅ Right Arm Rep Counted! Total: 5
✅ Left Arm Rep Counted! Total: 10
✅ Right Arm Rep Counted! Total: 6
✅ Left Arm Rep Counted! Total: 11
✅ Right Arm Rep Counted! Total: 7
✅ Left Arm Rep Counted! Total: 12
✅ Right Arm Rep Counted! Total: 8
✅ Left Arm Rep Counted! Total: 13
✅ Saved 21 reps of Side Reach (Single Arm Counting) to exercise_log.csv


: 